In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Load data

In [139]:
df=pd.read_csv('loan_2015.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (74,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
df.tail() 
# last two rows are invalid

,delinqamnt,openrv12m,numactvbctl,pubrec,memberid,purpose,numiltl,loanamnt,numtl120dpd2m,secappnumrevaccts,...,mthssincercntil,annualinc,annualincjoint,mthssincerecentrevoldelinq,accnowdelinq,chargeoffwithin12mths,grade,mthssincelastmajorderog,issued,loanstatus
421092,0.0,NaN,4.0,1.0,NaN,debt_consolidation,12.0,13000.0,0.0,NaN,...,NaN,35000.0,NaN,NaN,0.0,0.0,D,NaN,Jan-2015,Charged Off
421093,0.0,NaN,5.0,0.0,NaN,debt_consolidation,3.0,12000.0,0.0,NaN,...,NaN,64400.0,NaN,NaN,0.0,0.0,E,22.0,Jan-2015,Charged Off
421094,0.0,NaN,5.0,1.0,NaN,credit_card,1.0,20000.0,0.0,NaN,...,NaN,100000.0,NaN,NaN,0.0,0.0,B,NaN,Jan-2015,Fully Paid
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df.shape[0]

421097

In [142]:
df=df.drop([421095,421096])

# classify features

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421095 entries, 0 to 421094
Columns: 105 entries, delinqamnt to loanstatus
dtypes: float64(84), object(21)
memory usage: 340.5+ MB


In [143]:
col_nan=[column for column in df.columns if df[column].nunique()==0]
col_cate=[column for column in df.columns if df[column].dtype == 'object' and df[column].nunique()!=0]
col_num=[column for column in df.columns if df[column].dtype == 'float64' and df[column].nunique()!=0]
col_nouse=[column for column in df.columns if df[column].nunique(dropna=False)==1 and df[column].nunique()!=0]

In [144]:
df.shape

(421095, 105)

In [145]:
len(col_cate),len(col_num)

(20, 72)

# category features

In [71]:
df[col_cate]

,purpose,disbursementmethod,verificationstatus,addrstate,emplength,applicationtype,subgrade,initialliststatus,zipcode,homeownership,earliestcrline,desc,verificationstatusjoint,revolutil,term,intrate,emptitle,grade,issued,loanstatus
0,debt_consolidation,Cash,Not Verified,VA,10+ years,Individual,D3,f,231xx,OWN,Sep-1993,NaN,NaN,91.3%,60 months,17.27%,Teacher,D,Dec-2015,Current
1,debt_consolidation,Cash,Not Verified,AL,10+ years,Individual,B4,w,350xx,MORTGAGE,Aug-2001,NaN,NaN,69.8%,60 months,10.78%,Senior Buyer,B,Dec-2015,Current
2,debt_consolidation,Cash,Verified,MA,9 years,Individual,D3,w,023xx,OWN,Apr-2003,NaN,NaN,32.6%,36 months,17.27%,Department Manager,D,Dec-2015,Charged Off
3,debt_consolidation,Cash,Not Verified,CA,4 years,Individual,B2,w,958xx,MORTGAGE,Sep-2006,NaN,NaN,34.6%,36 months,9.17%,Registered Veterinary Technitian,B,Dec-2015,Fully Paid
4,debt_consolidation,Cash,Source Verified,GA,4 years,Individual,C3,w,300xx,RENT,Oct-1987,NaN,NaN,68.4%,36 months,13.44%,Veterinary Tecnician,C,Dec-2015,Fully Paid
5,credit_card,Cash,Not Verified,IL,6 years,Individual,C5,w,606xx,RENT,Jun-2007,NaN,NaN,84.5%,36 months,14.85%,GM,C,Dec-2015,Fully Paid
6,small_business,Cash,Not Verified,SD,10+ years,Individual,C1,w,577xx,MORTGAGE,Dec-1999,NaN,NaN,19.2%,36 months,11.99%,Engineer,C,Dec-2015,Fully Paid
7,debt_consolidation,Cash,Not Verified,IL,2 years,Individual,C4,w,604xx,RENT,May-2002,NaN,NaN,52.3%,36 months,13.99%,Software Testing Analyst,C,Dec-2015,Charged Off
8,debt_consolidation,Cash,Not Verified,NH,5 years,Individual,A4,w,030xx,MORTGAGE,Mar-2001,NaN,NaN,33.2%,36 months,7.49%,Systems Support Engineer,A,Dec-2015,Fully Paid
9,home_improvement,Cash,Not Verified,CT,10+ years,Individual,C2,w,064xx,OWN,Jan-1996,NaN,NaN,43.9%,60 months,12.88%,Unix System Administrator,C,Dec-2015,Current


In [146]:
print(df['term'].value_counts())

 36 months    283173
 60 months    137922
Name: term, dtype: int64


In [147]:
col_nouse

['disbursementmethod']

# get target

In [148]:
df['loanstatus'] = df.loanstatus.map({"Charged Off": 1, "Fully Paid": 0, "Default": 1})

In [149]:
train=df[df['term']==' 36 months']

In [ ]:
select_feature=col_cate+col_num
for item in col_nouse:
    select_feature.remove(item)
col_num.remove('fundedamnt')

In [151]:
col_nan

['memberid',
 'secappnumrevaccts',
 'secapprevolutil',
 'secappchargeoffwithin12mths',
 'secappcollections12mthsexmed',
 'secappinqlast6mths',
 'secappmortacc',
 'secappopenactil',
 'secappopenacc',
 'secappmthssincelastmajorderog',
 'secappearliestcrline',
 'revolbaljoint',
 'id']

In [152]:
col_cate.remove('loanstatus')
col_cate.remove('issued')

# desc, verificationstatusjoint feature as 1/0

In [153]:
train['desc']=(train['desc'].isnull()==False)*1

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [221]:
train['verificationstatusjoint']=(train['verificationstatusjoint'].isnull()==False)*1

In [154]:
train['desc'].value_counts()

0    283134
1        39
Name: desc, dtype: int64

In [155]:
select_feature=col_cate+col_num
len(select_feature)

89

In [156]:
train=train[select_feature+['loanstatus','issued']]

In [157]:
col_cate.remove('term')

In [158]:
train.loanstatus.value_counts(normalize=True)

0.0    0.850605
1.0    0.149395
Name: loanstatus, dtype: float64

# data in time split (Oct~Dec as test)

In [160]:
issued=list(train['issued'].unique())
issued

['Dec-2015',
 'Nov-2015',
 'Oct-2015',
 'Sep-2015',
 'Aug-2015',
 'Jul-2015',
 'Jun-2015',
 'May-2015',
 'Apr-2015',
 'Mar-2015',
 'Feb-2015',
 'Jan-2015']

In [168]:
train['train_flag']=train.issued.apply(lambda x: 0 if x in issued[:3] else 1)
train.train_flag.value_counts()

1    194363
0     88810
Name: train_flag, dtype: int64

# feature engineering

## earliestcrline gap

In [174]:
train.earliestcrline.unique()[:5]

array(['Apr-2003', 'Sep-2006', 'Oct-1987', 'Jun-2007', 'Dec-1999'],
      dtype=object)

In [171]:
dict_month={'Jan':11,'Feb':10,'Mar':9,'Apr':8, 'May':7, 'Jun':6, 'Jul':5, 'Aug':4, 'Sep':3, 'Oct':2, 'Nov':1, 'Dec':0}

In [178]:
train['gap_month_crline']=train.earliestcrline.apply(lambda x: (2015-int(x.split('-')[1]))*12+
                                            dict_month[x.split('-')[0]])

In [180]:
train['gap_month_crline'][:5]

2    152
3    111
4    338
5    102
6    192
Name: gap_month_crline, dtype: int64

In [181]:
col_num.append('gap_month_crline')

In [182]:
col_cate.remove('earliestcrline')

In [183]:
len(col_cate),len(col_num)

(16, 72)

## emplength to numeric feature

In [186]:
print(df['emplength'].isnull().sum())
train['emplength'].unique()

23817


array(['9 years', '4 years', '6 years', '10+ years', '2 years', '5 years',
       '3 years', '< 1 year', '1 year', '8 years', nan, '7 years'],
      dtype=object)

In [187]:
train['emplength'].replace('< 1 year','0',inplace=True)
train['emplength'].replace(to_replace='[^0-9]+',value='',inplace=True,regex=True)
train.emplength.fillna(-999,inplace=True)
train.emplength=train.emplength.astype('int')

In [189]:
train.emplength.value_counts()

 10     89689
 2      25889
 0      24091
 3      23110
 1      19240
-999    18970
 5      16899
 4      16756
 8      14270
 7      12163
 6      11182
 9      10914
Name: emplength, dtype: int64

In [196]:
col_num.append('emplength')
col_cate.remove('emplength')
len(col_cate),len(col_num)

(15, 73)

##  intrate, revolutil---->% to numeric feature

In [ ]:
(lambda x: float(x.replace('%','')))

In [204]:
train.intrate=train.intrate.apply(lambda x: float(x.replace('%','')))
train.revolutil=train.revolutil.apply(lambda x: str(x).replace('%','')).astype('float')

In [206]:
col_cate.remove('intrate')
col_num.append('intrate')
col_cate.remove('revolutil')
col_num.append('revolutil')

In [207]:
len(col_cate),len(col_num)

(13, 75)

## Ordinal feature encoding : grade, subgrade

In [208]:
dict_grade = {"A": 1, 
            "B": 2, 
            "C": 3, 
            "D": 4, 
            "E": 5,
            "F": 6, 
            "G": 7}

In [209]:
train.grade=train.grade.map(dict_grade)

In [211]:
train.subgrade=train.subgrade.apply(lambda x: (dict_grade[x[0]]-1)*5+int(x[1]))

In [212]:
col_cate.remove('grade')
col_num.append('grade')
col_cate.remove('subgrade')
col_num.append('subgrade')
len(col_cate),len(col_num)

(11, 77)

## High cardinality feature encoding

In [222]:
col_cate.remove('desc')
col_num.append('desc')
col_cate.remove('verificationstatusjoint')
col_num.append('verificationstatusjoint')
len(col_cate),len(col_num)

(9, 79)

In [224]:
col_cate.remove('disbursementmethod')
len(col_cate),len(col_num)

(8, 79)

In [226]:
train.zipcode=train.zipcode.apply(lambda x: int(x[:3]))
zipcode_freq = train.groupby("zipcode").size().reset_index()
zipcode_freq.columns = ["zipcode", "zipcode_freq"]

In [227]:
train = pd.merge(train, zipcode_freq, how = "left", on = "zipcode")

In [230]:
col_cate.remove('zipcode')
col_num.append('zipcode')
col_num.append('zipcode_freq')
len(col_cate), len(col_num)

(8, 81)

## emptitle

In [233]:
emptitle_freq = train.groupby("emptitle").size().reset_index()
emptitle_freq.columns = ["emptitle", "emptitle_freq"]
train = pd.merge(train, emptitle_freq, how = "left", on = "emptitle")

In [234]:
col_cate.remove('emptitle')
col_num.append('emptitle_freq')
len(col_cate), len(col_num)

(6, 82)

## address state

In [ ]:
addrstate_freq = train.groupby("addrstate").size().reset_index()
addrstate_freq.columns = ["addrstate", "addrstate_freq"]
train = pd.merge(train, addrstate_freq, how = "left", on = "addrstate")

In [235]:
col_cate.remove('addrstate')
col_num.append('addrstate_freq')
len(col_cate), len(col_num)

(5, 83)

## one hot encoding

In [236]:
col_cate

['purpose',
 'verificationstatus',
 'applicationtype',
 'initialliststatus',
 'homeownership']

In [237]:
dummy_feature = ["homeownership", "verificationstatus", 'applicationtype', "purpose", "initialliststatus"]

In [238]:
df_dummy = pd.get_dummies(train[dummy_feature])

In [242]:
OHE_feature=list(df_dummy.columns.values)

In [241]:
train=pd.concat([train,df_dummy], axis=1)

In [243]:
len(OHE_feature)

25

In [244]:
train.shape

(283173, 120)

In [245]:
for column in dummy_feature:
    col_cate.remove(column)

In [247]:
select_feature=col_num+OHE_feature
len(select_feature)

108

# xgboost

In [ ]:
## frequency encoding; leave one out encoding; emtiltle (case sensitive; overall category: manager)
# secondary model: insights